# 1D Gaussian Process Fitting

This notebook fits a 1D Gaussian Process (GP) to `data.csv` with interactive controls for:

- choosing `x` and `f(x)` columns
- selecting a kernel
- tuning kernel and model hyperparameters
- visualizing fit and uncertainty

Expected CSV headers:

- `bumper hub (cm)`
- `main`
- `back`


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (
    ConstantKernel,
    WhiteKernel,
    RBF,
    Matern,
    RationalQuadratic,
    ExpSineSquared,
    DotProduct,
)

import ipywidgets as widgets
from IPython.display import display

plt.style.use("seaborn-v0_8-whitegrid")


In [2]:
CSV_PATH = Path("data.csv")

if not CSV_PATH.exists():
    raise FileNotFoundError(
        f"Could not find {CSV_PATH.resolve()}\n"
        "Place data.csv in the same folder as this notebook."
    )

df = pd.read_csv(CSV_PATH)
# Normalize accidental whitespace in headers
df.columns = [c.strip() for c in df.columns]

print("Loaded columns:", list(df.columns))
display(df.head())

numeric_columns = [
    c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])
]

if len(numeric_columns) < 2:
    raise ValueError("Need at least two numeric columns to fit 1D GP.")

print("Numeric columns available for fitting:", numeric_columns)


Loaded columns: ['bumper-hub (cm)', 'main', 'back']


,bumper-hub (cm),main,back
0,181,161,177
1,272,148,220
2,309,143,263
3,394,120,360


Numeric columns available for fitting: ['bumper-hub (cm)', 'main', 'back']


In [3]:
def build_base_kernel(kernel_name, p):
    if kernel_name == "RBF":
        return RBF(length_scale=p["rbf_length_scale"], length_scale_bounds=(1e-5, 1e5))

    if kernel_name == "Matern":
        return Matern(
            length_scale=p["matern_length_scale"],
            nu=p["matern_nu"],
            length_scale_bounds=(1e-5, 1e5),
        )

    if kernel_name == "RationalQuadratic":
        return RationalQuadratic(
            length_scale=p["rq_length_scale"],
            alpha=p["rq_alpha"],
            length_scale_bounds=(1e-5, 1e5),
            alpha_bounds=(1e-5, 1e5),
        )

    if kernel_name == "ExpSineSquared":
        return ExpSineSquared(
            length_scale=p["ess_length_scale"],
            periodicity=p["ess_periodicity"],
            length_scale_bounds=(1e-5, 1e5),
            periodicity_bounds=(1e-5, 1e5),
        )

    if kernel_name == "DotProduct":
        return DotProduct(sigma_0=p["dp_sigma0"], sigma_0_bounds=(1e-5, 1e5))

    raise ValueError(f"Unknown kernel: {kernel_name}")


def build_kernel(kernel_name, p):
    k = build_base_kernel(kernel_name, p)

    if p["use_constant"]:
        k = ConstantKernel(
            constant_value=p["constant_value"],
            constant_value_bounds=(1e-5, 1e5),
        ) * k

    if p["add_white"]:
        k = k + WhiteKernel(
            noise_level=p["noise_level"],
            noise_level_bounds=(1e-8, 1e2),
        )

    return k


In [4]:
# Global controls
x_dropdown = widgets.Dropdown(options=numeric_columns, description="x:")
y_dropdown = widgets.Dropdown(options=numeric_columns, description="f(x):")
if len(numeric_columns) > 1:
    y_dropdown.value = numeric_columns[1]

kernel_dropdown = widgets.Dropdown(
    options=["RBF", "Matern", "RationalQuadratic", "ExpSineSquared", "DotProduct"],
    value="RBF",
    description="Kernel:",
)

normalize_y_widget = widgets.Checkbox(value=True, description="normalize_y")
alpha_widget = widgets.FloatLogSlider(
    value=1e-8, base=10, min=-12, max=-2, step=0.2, description="alpha:", readout_format=".1e"
)
n_restarts_widget = widgets.IntSlider(value=3, min=0, max=20, step=1, description="restarts:")
n_pred_widget = widgets.IntSlider(value=300, min=100, max=2000, step=50, description="grid pts:")

# Shared kernel modifiers
use_constant_widget = widgets.Checkbox(value=True, description="Use ConstantKernel")
constant_value_widget = widgets.FloatLogSlider(
    value=1.0, base=10, min=-3, max=3, step=0.1, description="constant:"
)
add_white_widget = widgets.Checkbox(value=True, description="Add WhiteKernel")
noise_level_widget = widgets.FloatLogSlider(
    value=1e-3, base=10, min=-8, max=0, step=0.1, description="noise:"
)

# Kernel-specific controls
rbf_length_scale_widget = widgets.FloatLogSlider(
    value=1.0, base=10, min=-3, max=3, step=0.1, description="length_scale:"
)

matern_length_scale_widget = widgets.FloatLogSlider(
    value=1.0, base=10, min=-3, max=3, step=0.1, description="length_scale:"
)
matern_nu_widget = widgets.Dropdown(options=[0.5, 1.5, 2.5], value=1.5, description="nu:")

rq_length_scale_widget = widgets.FloatLogSlider(
    value=1.0, base=10, min=-3, max=3, step=0.1, description="length_scale:"
)
rq_alpha_widget = widgets.FloatLogSlider(
    value=1.0, base=10, min=-3, max=3, step=0.1, description="alpha:"
)

ess_length_scale_widget = widgets.FloatLogSlider(
    value=1.0, base=10, min=-3, max=3, step=0.1, description="length_scale:"
)
ess_periodicity_widget = widgets.FloatLogSlider(
    value=1.0, base=10, min=-3, max=3, step=0.1, description="periodicity:"
)

dp_sigma0_widget = widgets.FloatLogSlider(
    value=1.0, base=10, min=-3, max=3, step=0.1, description="sigma0:"
)

kernel_param_box = widgets.VBox()
output = widgets.Output()


def kernel_specific_widgets(name):
    if name == "RBF":
        return [rbf_length_scale_widget]
    if name == "Matern":
        return [matern_length_scale_widget, matern_nu_widget]
    if name == "RationalQuadratic":
        return [rq_length_scale_widget, rq_alpha_widget]
    if name == "ExpSineSquared":
        return [ess_length_scale_widget, ess_periodicity_widget]
    if name == "DotProduct":
        return [dp_sigma0_widget]
    return []


def refresh_kernel_param_box(*_):
    kernel_param_box.children = [
        widgets.HTML("<b>Kernel parameters</b>"),
        *kernel_specific_widgets(kernel_dropdown.value),
        widgets.HTML("<b>Kernel modifiers</b>"),
        use_constant_widget,
        constant_value_widget,
        add_white_widget,
        noise_level_widget,
    ]


kernel_dropdown.observe(refresh_kernel_param_box, names="value")
refresh_kernel_param_box()


In [5]:
# Store latest optimized fit for code generation
last_fit = {}


def collect_params():
    return {
        "rbf_length_scale": rbf_length_scale_widget.value,
        "matern_length_scale": matern_length_scale_widget.value,
        "matern_nu": matern_nu_widget.value,
        "rq_length_scale": rq_length_scale_widget.value,
        "rq_alpha": rq_alpha_widget.value,
        "ess_length_scale": ess_length_scale_widget.value,
        "ess_periodicity": ess_periodicity_widget.value,
        "dp_sigma0": dp_sigma0_widget.value,
        "use_constant": use_constant_widget.value,
        "constant_value": constant_value_widget.value,
        "add_white": add_white_widget.value,
        "noise_level": noise_level_widget.value,
    }


def fit_and_plot(_=None):
    with output:
        output.clear_output(wait=True)

        x_col = x_dropdown.value
        y_col = y_dropdown.value

        if x_col == y_col:
            print("Please choose different columns for x and f(x).")
            return

        d = df[[x_col, y_col]].dropna().copy()
        d = d.sort_values(x_col)

        if len(d) < 3:
            print("Not enough valid data points after dropping NaNs.")
            return

        X = d[[x_col]].to_numpy()
        y = d[y_col].to_numpy()

        params = collect_params()
        kernel_name = kernel_dropdown.value
        kernel = build_kernel(kernel_name, params)

        gp = GaussianProcessRegressor(
            kernel=kernel,
            alpha=alpha_widget.value,
            normalize_y=normalize_y_widget.value,
            n_restarts_optimizer=n_restarts_widget.value,
            random_state=0,
        )

        gp.fit(X, y)
        optimized_lml = gp.log_marginal_likelihood(gp.kernel_.theta)

        x_min, x_max = float(X.min()), float(X.max())
        pad = 0.05 * (x_max - x_min if x_max > x_min else 1.0)
        x_pred = np.linspace(x_min - pad, x_max + pad, n_pred_widget.value).reshape(-1, 1)

        y_mean, y_std = gp.predict(x_pred, return_std=True)

        fig, ax = plt.subplots(figsize=(9, 5))
        ax.scatter(X[:, 0], y, s=28, alpha=0.8, label="data")
        ax.plot(x_pred[:, 0], y_mean, lw=2.2, label="GP mean (optimized kernel)")
        ax.fill_between(
            x_pred[:, 0],
            y_mean - 1.96 * y_std,
            y_mean + 1.96 * y_std,
            alpha=0.2,
            label="95% CI",
        )

        ax.set_xlabel(x_col)
        ax.set_ylabel(y_col)
        ax.set_title(f"1D GP Fit: {y_col} vs {x_col}")
        ax.legend()
        plt.show()

        print("Optimized kernel:")
        print(gp.kernel_)
        print(f"Log-marginal likelihood: {optimized_lml:.4f}")
        print("Plot curve and uncertainty are generated from this optimized kernel.")

        # Cache optimized fit for interpolation code generation.
        last_fit.clear()
        last_fit.update({
            "gp": gp,
            "x_col": x_col,
            "y_col": y_col,
            "x_min": x_min,
            "x_max": x_max,
            "optimized_kernel": str(gp.kernel_),
            "log_marginal_likelihood": float(optimized_lml),
        })


fit_button = widgets.Button(description="Fit & Plot", button_style="primary", icon="line-chart")
fit_button.on_click(fit_and_plot)

ui = widgets.VBox([
    widgets.HTML("<h3>GP 1D Fitting Controls</h3>"),
    widgets.HBox([x_dropdown, y_dropdown]),
    kernel_dropdown,
    kernel_param_box,
    widgets.HTML("<b>Model parameters</b>"),
    normalize_y_widget,
    alpha_widget,
    n_restarts_widget,
    n_pred_widget,
    fit_button,
])

display(ui, output)

# Run once with defaults
fit_and_plot()

Output()

In [7]:
# Generate Java interpolation map code from the latest optimized GP fit
codegen_output = widgets.Output()

map_name_dropdown = widgets.Dropdown(
    options=["distanceToBackwheelMap", "backwheelToFlywheelSpeedMap"],
    value="distanceToBackwheelMap",
    description="Map var:",
)

num_samples_widget = widgets.IntSlider(
    value=25,
    min=5,
    max=300,
    step=1,
    description="# samples:",
)


def generate_map_code(_=None):
    with codegen_output:
        codegen_output.clear_output(wait=True)

        if not last_fit:
            print("No optimized GP fit found yet. Run 'Fit & Plot' first.")
            return

        gp = last_fit["gp"]
        x_min = last_fit["x_min"]
        x_max = last_fit["x_max"]
        n = num_samples_widget.value

        x_samples = np.linspace(x_min, x_max, n).reshape(-1, 1)
        y_samples = gp.predict(x_samples)

        map_name = map_name_dropdown.value

        print("// Generated from optimized GP fit")
        print(f"// Fit: {last_fit['y_col']} vs {last_fit['x_col']}")
        print(f"// Optimized kernel: {last_fit['optimized_kernel']}")
        print(f"// Log-marginal likelihood: {last_fit['log_marginal_likelihood']:.4f}")
        print()

        for x_val, y_val in zip(x_samples[:, 0], y_samples):
            print(f"{map_name}.put({x_val:.4f}, {y_val:.4f});")


generate_button = widgets.Button(
    description="Generate",
    button_style="success",
    icon="code",
)
generate_button.on_click(generate_map_code)

codegen_ui = widgets.VBox([
    widgets.HTML("<h3>Interpolation Code Generation</h3>"),
    widgets.HTML(
        "Uses the latest optimized GP fit. Choose map variable name and sample density."
    ),
    map_name_dropdown,
    num_samples_widget,
    generate_button,
])

display(codegen_ui, codegen_output)

Output()